In [2]:
from src.inverse_astar_search.network import *
# from src.inverse_astar_search.pastar import *

from src.inverse_astar_search.pastar import heuristic_bounds, astar_path_heuristic_nodes\
    , set_heuristic_costs_nodes, set_heuristic_costs_edges, get_edge_attributes_labels,path_generator\
    , chosen_edges, path_length, paths_lengths, neighbors_path

from src.inverse_astar_search.logit import create_decision_matrix_from_observed_path\
    , logit_estimation, compute_edge_utility, logit_path_predictions, accuracy_pastar_predictions

# External modules
import numpy as np
import networkx as nx

In [4]:
def create_network_data(n_nodes, n_paths, theta_training):

    nodes_G = n_nodes

    # Adjacency matrix
    A = np.random.randint(0, 2, [nodes_G, nodes_G])

    # Assign random weight to edges
    W = random_edge_weights(A=A, limits=(1, 100))

    # Create networkX graph
    G = create_network(W)

    # Edges attributes

    # - Distance [km]
    nx.set_edge_attributes(G, values=nx.get_edge_attributes(G, 'weight'), name='distance')

    # - Cost ($)
    cost_edges = dict(zip(dict(G.edges).keys(), np.random.randint(0, 20, len(list(G.edges)))))
    nx.set_edge_attributes(G, values=cost_edges, name='cost')

    # - Travel time (mins)
    speed = 20  # km/hr
    travel_time_edges = dict(zip(dict(G.edges).keys(),
                                 np.round(60 / speed * np.array(list(nx.get_edge_attributes(G, 'distance').values())),
                                          1)))
    nx.set_edge_attributes(G, values=travel_time_edges, name='travel_time')

    # Utility at edges
    utility_edges = compute_edge_utility(G, theta=theta_training)
    nx.set_edge_attributes(G, utility_edges, name='utility')

    # Weight equals to utility
    weight_edges = {key: -val for key, val in nx.get_edge_attributes(G, 'utility').items()}
    nx.set_edge_attributes(G, values=weight_edges, name='weight')

    # Simulate observed paths
    observed_paths = path_generator(G = G, n_pairs=n_paths, attribute='weight')

    # Heuristic cost at edge level
    G = heuristic_bounds(G=G, observed_path=observed_paths[0])
    G = set_heuristic_costs_nodes(G)
    G = set_heuristic_costs_edges(G)

    # # Observed path (real observation)
    # print((observed_paths[0], path_length(G, path=observed_paths[0], attribute='distance')))

    # nx.get_node_attributes(G, 'f_bound_neighbor')
    # nx.get_node_attributes(G, 'h_bound_neighbor')
    # nx.get_node_attributes(G, 'h_bound_optimal')

    return G, observed_paths

# Preference parameters
theta_G_training = {'travel_time': -4, 'cost': -2} #Signs are not required to be negative
vot = theta_G_training['travel_time'] / theta_G_training['cost']

G_training,observed_paths = create_network_data(n_nodes = 10, n_paths = 50, theta_training = theta_G_training)

print(vot)

2.0
